# Laboratorium 4 - rekomendacje dla portali informacyjnych

## Przygotowanie

 * pobierz i wypakuj dataset: https://mind201910small.blob.core.windows.net/release/MINDsmall_train.zip
   * więcej możesz poczytać tutaj: https://learn.microsoft.com/en-us/azure/open-datasets/dataset-microsoft-news
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab4`
 * zainstaluj potrzebne biblioteki:
 `pip install nltk sklearn`

## Część 1. - przygotowanie danych

In [16]:
# importujemy wszystkie potrzebne pakiety

import codecs
from collections import defaultdict # mozesz uzyc zamiast zwyklego slownika, rozwaz wplyw na czas obliczen
import math
import re
from string import punctuation

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# mozesz uzyc do obliczania najbardziej podobnych tekstow zamiast liczenia "na piechote"
# ale pamietaj o dostosowaniu formatu danych
from sklearn.neighbors import NearestNeighbors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mateu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# definiujemy potrzebne zmienne

PATH = './MINDsmall_train'
STOPWORDS = set(stopwords.words('english'))

In [18]:
# wczytujemy metadane artykułów

def parse_news_entry(entry):
    news_id, category, subcategory, title, abstract = entry.split('\t')[:5]
    return {
        'news_id': news_id,
        'category': category,
        'subcategory': subcategory,
        'title': title,
        'abstract': abstract
    }

def get_news_metadata():
    with codecs.open(f'{PATH}/news.tsv', 'r', 'UTF-8') as f:
        raw = [x for x in f.read().split('\n') if x]
        parsed_entries = [parse_news_entry(entry) for entry in raw]
        return {x['news_id']: x for x in parsed_entries}

news = get_news_metadata()
news_ids = sorted(list(news.keys()))
news_indices = {x[1]: x[0] for x in enumerate(news_ids)}
print(len(news))

51282


## Część 2. - TF-IDF

In [20]:
# normalizujemy teksty na potrzeby dalszego przetwarzania

def preprocess_text(text):
    # zamieniamy wszystkie ciagi bialych znakow na pojedyncze spacje
    text= ' '.join(text.split())
    # podmieniamy wszystkie wielkie litery
    text = text.lower()
    # usuwamy znaki interpunkcyjne
    # usuwamy wszystkie liczby
    whitelist = set('abcdefghijklmnopqrstuvwxyz ')
    text = ''.join(filter(whitelist.__contains__, text))

    # dzielimy na tokeny
    tokens = text.split(" ")
    tokens = [token for token in tokens if token not in STOPWORDS]
    # usuwamy stopwords
    return tokens
def stem_texts(corpus):
    stemmer = SnowballStemmer(language="english")
    return [[stemmer.stem(word) for word in preprocess_text(text)] for text in corpus]

texts = [news[news_id]['abstract'] for news_id in news_ids]
stemmed_texts = stem_texts(texts)

In [21]:
# porownajmy teksty przed i po przetworzeniu

print(texts[2] + '\n')
print(' '.join(stemmed_texts[2]))

"I think we have a really good team, and a team that can really do some special, good things because that group is very close in there." - Brian Schmetzer

think realli good team team realli special good thing group close  brian schmetzer


In [6]:
# tworzymy liste wszystkich slow w korpusie

def get_all_words_sorted(corpus):
    wordlist = set()
    for text in corpus:
        for word in text:
            wordlist.add(word)
    return sorted(wordlist)


wordlist = get_all_words_sorted(stemmed_texts)
word_indices = {x[1]: x[0] for x in enumerate(wordlist)}
print(len(wordlist))

41634


In [23]:
# obliczamy liczbe tekstow, w ktorych wystapilo kazde ze slow
# pamietaj, ze jesli slowo wystapilo w danym tekscie wielokrotnie, to liczymy je tylko raz

def get_document_frequencies(corpus, wordlist):
    # return {word -> count}
    document_frequency = defaultdict(int)
    # for word in wordlist:
    #     document_frequency[word]=0
    
    for text in corpus:
        for word in set(text):
            document_frequency[word]+=1
    
    return document_frequency
    

document_frequency = get_document_frequencies(stemmed_texts, wordlist)
document_frequency

defaultdict(int,
            {'fartlek': 1,
             'capit': 196,
             '': 22930,
             'wretch': 1,
             'conundrum': 2,
             'nation': 1629,
             'somewher': 37,
             'confer': 384,
             'game': 2866,
             'concern': 328,
             'remind': 107,
             'one': 4098,
             'run': 1096,
             'altuv': 35,
             'assembl': 71,
             'teammat': 143,
             'airplan': 42,
             'washington': 810,
             'team': 2360,
             'crop': 43,
             'verland': 55,
             'loss': 797,
             'justin': 177,
             'wednesday': 2140,
             'two': 3113,
             'jose': 83,
             'sky': 146,
             'playerson': 2,
             'prestig': 4,
             'meet': 581,
             'astro': 404,
             'eastern': 167,
             'within': 232,
             'encourag': 81,
             'veteran': 686,
             'regul

In [24]:
# obliczamy liczbe wystapien kazdego slowa w kazdym tekscie

def get_term_frequencies(corpus, news_indices):
    # return {news_id -> {word -> count}}
    term_frequency = dict()

    for news_id, i in news_indices.items():
        counter = defaultdict(int)
        for word in corpus[i]:
            counter[word]+=1
        term_frequency[news_id]=counter

    return term_frequency

term_frequency = get_term_frequencies(stemmed_texts, news_indices)

In [25]:
# sprawdzmy wyniki

term_frequency[news_ids[2]]

defaultdict(int,
            {'think': 1,
             'realli': 2,
             'good': 2,
             'team': 2,
             'special': 1,
             'thing': 1,
             'group': 1,
             'close': 1,
             '': 1,
             'brian': 1,
             'schmetzer': 1})

In [26]:
# obliczamy metryke tf_idf

def calculate_tf_idf(term_frequency, document_frequency, corpus_size):
    # return {news_id -> {word -> tf_idf}}
    tf_idf = dict()
    for news_id, term_counter in term_frequency.items():
        word_w=dict()
        for word, tf in term_counter.items():
            word_w[word]=tf*math.log(corpus_size/document_frequency[word])
        tf_idf[news_id]=word_w

    return tf_idf
tf_idf = calculate_tf_idf(term_frequency, document_frequency, len(news_ids))

In [27]:
# sprawdzmy wyniki

tf_idf[news_ids[2]]

{'think': 4.360459856758821,
 'realli': 9.707261090572182,
 'good': 8.085400658139525,
 'team': 6.157356388748834,
 'special': 4.765161897298483,
 'thing': 4.110503432421125,
 'group': 4.261685870235309,
 'close': 3.9244235881453897,
 '': 0.8048937165421228,
 'brian': 5.624739267315748,
 'schmetzer': 9.458800731274183}

## Część 3. - Podobieństwo tekstów

In [43]:
# obliczmy odleglosc miedzy dwoma artykulami
# przetestuj rozne metryki odleglosci i wybierz najlepsza
def euclides(tf_idf, id1, id2):
    words = set(tf_idf[id1].keys() | tf_idf[id2].keys())
    dist=0
    for word in words:
        x=tf_idf[id1].get(word,0)
        y=tf_idf[id2].get(word,0)
        dist+=(x-y)**2
    return math.sqrt(dist)

def taxi(tf_idf, id1, id2):
    words = set(tf_idf[id1].keys() | tf_idf[id2].keys())
    dist=0
    for word in words:
        x=tf_idf[id1].get(word,0)
        y=tf_idf[id2].get(word,0)
        dist+=abs(x-y)
    return dist

def max_metric(tf_idf, id1, id2):
    words = set(tf_idf[id1].keys() | tf_idf[id2].keys())
    dist=float('-inf')
    for word in words:
        x=tf_idf[id1].get(word,0)
        y=tf_idf[id2].get(word,0)
        dist=max(dist,abs(x-y))
    return dist


def calculate_distance(tf_idf, id1, id2):
    eucli=euclides(tf_idf, id1, id2)
    taxi_c = taxi(tf_idf, id1, id2)
    max_c = max_metric(tf_idf, id1, id2)
    # print("euklides: ",eucli)
    # print("taxi: ",taxi_c)
    # print("max: ",max_c)
    return eucli


calculate_distance(tf_idf, news_ids[2], news_ids[1])

42.47526391000354

In [45]:
# wyznaczmy k najpodobniejszych tekstow do danego
# pamietaj o odpowiedniej kolejnosci sortowania w zaleznosci od wykorzystanej metryki
# pamietaj, zeby wsrod podobnych tekstow nie bylo danego

def get_k_most_similar_news(tf_idf, n_id, k):
    distances = dict()
    for news_id in tf_idf.keys():
        if news_id != n_id:
            distances[news_id] = calculate_distance(tf_idf,news_id, n_id)
    distances = sorted(distances.items(), key=lambda x: x[1])[:k]

    return dict(distances).keys()

def print_k_most_similar_news(tf_idf, n_id, k, corpus, news_indices):
    similar = get_k_most_similar_news(tf_idf, n_id, k)
    print(f'id: {n_id}, text: {corpus[news_indices[n_id]]}')
    print(f'\n{k} most similar:')
    for s_id in similar:
        print(f'\nid: {s_id}, text: {corpus[news_indices[s_id]]}')

print_k_most_similar_news(tf_idf, news_ids[42337], 5, texts, news_indices)

id: N58544, text: A MAN claims he has created a car that might solve the world's traffic congestion problems. Rick Woodbury from Spokane, Washington USA, is the president, founder and sole employee of 'Commuter Cars.' The carmaker's flagship model is the 2005 super slim two-seater Tango T600, a high-performance electric car that preceded Tesla. Rick told BTV: "I started this company 21 years ago   it was based on an idea that I came up with in 1982." He was inspired by the shocking traffic congestion he had to face on a daily basis. "I used to drive a Porsche from Beverly Hills to Hermosa Beach every day and the traffic was horrendous," explained Rick. What really made Rick think about a solution was the fact that in most of the cars he would see in his commute were occupied by lone drivers. "I noticed that everybody around me was a single occupant in a car, taking up the whole lane," Rick said. Living and working in Los Angeles also helped inspire Rick's unique creation. "I thought, e